## Imports & Definitions

In [1]:
import warnings

warnings.filterwarnings(
    "ignore",
    message="torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.",
)

warnings.filterwarnings(
    "ignore",
    message="torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.",
)


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import sys
sys.path.append("../../utils")
from definitions import *

In [13]:
! ls ../../../../

LongLoRA-diploma-research
OpenBookQA
YandexGPT-api-call_ru.ipynb
cache
dataflow_en.ipynb
gpt-week
long_context_LLMs
modelcache
nlp_course
venv
view_machine.ipynb
wandb
wandb_try.ipynb


In [4]:
CACHE_DIR = Path("../../../../cache/")
DATASET_DIR = Path("/home/jupyter/mnt/datasets/diplomas/russian_dataset/")

## Example of Fine-Tuning

### Helper imports & definitions

In [5]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

2024-05-08 19:23:30.816489: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
    "prompt_no_input_llama2":(
        "[INST] <<SYS>>\n"
        "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\n"
        "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n"
        "<</SYS>> \n\n {instruction} [/INST]"
    ),
    "prompt_input_llama2": (
        "[INST] <<SYS>>\n"
        "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\n"
        "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n"
        "<</SYS>> \n\n {instruction} \n{input} [/INST]"
    ),
    "prompt_llama2": "[INST]{instruction}[/INST]",
    "prompt_input_diploma_special":(
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\nBelow is a diploma text. Your task is to generate abstract of this diploma.\n\n### Input:\n{input}\n\n### Response:"
    ),
}

In [7]:
from typing import Dict, Optional, Sequence

def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )


def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [8]:
from torch.utils.data import Dataset
import logging

class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer, nrows: int, diploma_prefix_len: int):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        data_table = pd.read_csv(data_path, nrows=nrows)

        logging.warning("Formatting inputs...")

        prompt_input_diploma = PROMPT_DICT["prompt_input_diploma_special"]
        sources = [
            prompt_input_diploma.format(input=diploma[:diploma_prefix_len])
            for diploma in data_table["diploma"]
        ]

        targets = [f"{abstract}{tokenizer.eos_token}" for abstract in data_table["abstract"]]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

### Downloading model & tokenizer

In [9]:
model_name = LLAMA_2_7B

In [10]:
model = AutoModelForCausalLM.from_pretrained(HUGGINGFACE_MODEL_TO_REPO[model_name], cache_dir=CACHE_DIR, device_map='auto')

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.67s/it]


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [13]:
MODEL_MAX_LENGTH = 16384

In [14]:
tokenizer = AutoTokenizer.from_pretrained(
    HUGGINGFACE_MODEL_TO_REPO[model_name], 
    cache_dir=CACHE_DIR, 
    model_max_length=MODEL_MAX_LENGTH,
    padding_side="right",
    use_fast=True)

In [15]:
tokenizer.model_max_length

16384

### Add new tokens

In [16]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"

def smart_tokenizer_and_embedding_resize(
    special_tokens_dict: Dict,
    tokenizer: transformers.PreTrainedTokenizer,
    model: transformers.PreTrainedModel,
):
    """Resize tokenizer and embedding.

    Note: This is the unoptimized version that may make your embedding size not be divisible by 64.
    """
    num_new_tokens = tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

    if num_new_tokens > 0:
        input_embeddings = model.get_input_embeddings().weight.data
        output_embeddings = model.get_output_embeddings().weight.data

        input_embeddings_avg = input_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)
        output_embeddings_avg = output_embeddings[:-num_new_tokens].mean(dim=0, keepdim=True)

        input_embeddings[-num_new_tokens:] = input_embeddings_avg
        output_embeddings[-num_new_tokens:] = output_embeddings_avg

special_tokens_dict = dict()
if tokenizer.pad_token is None:
    special_tokens_dict["pad_token"] = DEFAULT_PAD_TOKEN
if tokenizer.eos_token is None:
    special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
if tokenizer.bos_token is None:
    special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
if tokenizer.unk_token is None:
    special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN

smart_tokenizer_and_embedding_resize(
    special_tokens_dict=special_tokens_dict,
    tokenizer=tokenizer,
    model=model,
)

Using pad_token, but it is not set yet.


### Load train/val/test datasets

In [17]:
train_dataset = SupervisedDataset(DATASET_DIR.joinpath("russian_dataset_train.csv"), tokenizer, nrows=720, diploma_prefix_len=1000)

In [18]:
val_dataset = SupervisedDataset(DATASET_DIR.joinpath("russian_dataset_val.csv"), tokenizer,  nrows=720, diploma_prefix_len=1000)

In [49]:
test_dataset = SupervisedDataset(DATASET_DIR.joinpath("russian_dataset_test.csv"), tokenizer,  nrows=10, diploma_prefix_len=1000)

### Ensure that diploma_prefix_len is ok

In [20]:
list(set(test_dataset[9]["labels"].tolist()))[:30]

[2,
 517,
 24072,
 24585,
 23567,
 8211,
 531,
 26133,
 1046,
 2583,
 1561,
 24090,
 6687,
 14367,
 10786,
 2082,
 551,
 23082,
 8747,
 6188,
 6193,
 6195,
 1587,
 1086,
 3648,
 1604,
 25671,
 15432,
 1097,
 16970]

### View how dataset built

In [56]:
text_labels = tokenizer.decode(test_dataset[9]["input_ids"])
print(text_labels)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is a diploma text. Your task is to generate abstract of this diploma.

### Input:
Санкт-Петербургский государственный университет


АВРАМЕНКО Полина Андреевна
Выпускная квалификационная работа
Веб-туны как часть южнокорейской культуры в XXI веке (на примере романтических историй) 
Уровень образования: магистратура
Направление 58.04.01 «Востоковедение и африканистика»
Основная образовательная программа BM.5808 «Культура народов Азии и Африки (с изучением языков Азии и Африки)»


Научный руководитель:
доцент, Кафедра корееведения, Санкт-Петербургский государственный университет Гурьева Анастасия Александровна

Рецензент:
приглашенный преподаватель, Кафедра корееведения, Санкт-Петербургская школа социальных наук и востоковедения,
доцент, Санкт-Петербургский филиал федерального государственного автономного 

In [61]:
unignored_tokens = []
for token in test_dataset[9]["labels"]:
    if token != IGNORE_INDEX:
        unignored_tokens.append(token)
text_labels = tokenizer.decode(unignored_tokens)
print(text_labels)

Данная выпускная квалификационная работа посвящена одному из основных элементов, формирующих массовый культурный контент Республики Корея - веб-тунам (webtoon) – цифровым комиксам, появившимся в начале XXI века. Целью работы является выявление места веб-тунов в южнокорейской культуре, а также их культурной специфики. Актуальность исследования обусловлена тем, что в наши дни в Южной Корее к веб-тунам наблюдается повышенный интерес общества. В ходе исследования был собран, изучен и систематизирован материал об истории манхва как предшественника веб-тунов. Были рассмотрены этапы развития веб-тунов, причины популярности и основные характеристики. В качестве материала для исследования были выбраны и проанализированы три популярных южнокорейских веб-туна. Посредством анализа была выявлена специфика подачи материала и связь с культурой. Благодаря анализу удалось выявить взаимосвязь веб-тунов с традиционной литературой и ролью текста в корейской культуре и традиционным распределением ролей.</s

### Generating with raw model before learning

In [72]:
# inference train sample
model.eval()
prefix_len = np.sum(np.array(train_dataset[9]["labels"]) == IGNORE_INDEX)
prefix_tokens = train_dataset[9]["input_ids"][:prefix_len]
generated = model.generate(prefix_tokens.reshape((1, -1)).to(device))
generated_text = tokenizer.decode(generated.to('cpu').flatten())
print(generated_text)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is a diploma text. Your task is to generate abstract of this diploma.

### Input:
САНКТ-ПЕТЕРБУРГСКИЙ ГОСУДАРСТВЕННЫЙ
УНИВЕРСИТЕТ

Мануэль Антонио Агилар Ривера
Выпускная квалификационная работа
«Multiplication Operators in Cauchy-de Branges Spaces»
Уровень образования: магистратура
Направление 01.04.01 “Математика”
Основная образовательная программа BM.5832.2019
“Современная математика”
Научный руководитель:
Профессор, математико-механический факультет СПбГУ,
доктор физ.-мат. наук, профессор РАН
Баранов Антон Дмитриевич.
Рецензент:
Профессор, Факультет математики,
Автономный университет Мадрида, кандидат физ.-мат. наук
Якубович Дмитрий Владимирович
Санкт-Петербург
2021

Contents
Contents

2

Cauchy–de Branges spaces
3
Cauchy–de Branges spaces as Reproducing Kernel Hilbert spaces . 4
The Division Prope

In [62]:
prefix_len = np.sum(np.array(test_dataset[9]["labels"]) == IGNORE_INDEX)
prefix_len

443

In [63]:
model.eval()
prefix_tokens = test_dataset[9]["input_ids"][:prefix_len]
text = tokenizer.decode(prefix_tokens)
print(text)
print()
generated = model.generate(prefix_tokens.reshape((1, -1)).to(device))
generated_text = tokenizer.decode(generated.to('cpu').flatten())
print(generated_text)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is a diploma text. Your task is to generate abstract of this diploma.

### Input:
Санкт-Петербургский государственный университет


АВРАМЕНКО Полина Андреевна
Выпускная квалификационная работа
Веб-туны как часть южнокорейской культуры в XXI веке (на примере романтических историй) 
Уровень образования: магистратура
Направление 58.04.01 «Востоковедение и африканистика»
Основная образовательная программа BM.5808 «Культура народов Азии и Африки (с изучением языков Азии и Африки)»


Научный руководитель:
доцент, Кафедра корееведения, Санкт-Петербургский государственный университет Гурьева Анастасия Александровна

Рецензент:
приглашенный преподаватель, Кафедра корееведения, Санкт-Петербургская школа социальных наук и востоковедения,
доцент, Санкт-Петербургский филиал федерального государственного автономного 

### Train model

In [21]:
model_type = "llama" # default
if model_type == "gpt-neox":
    # added `dense` to match with llama as the basic LoRA would only target 'query_key_value'
    targets = ["query_key_value", "dense"]
else:
    targets=["q_proj", "k_proj", "v_proj", "o_proj"]

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=targets,
    lora_dropout=0,
    bias="none",
    task_type="CAUSAL_LM",
)

In [22]:
 peft_model = get_peft_model(model, config)

In [23]:
trainable_params = "embed,norm"

In [24]:
[p.requires_grad_() for n, p in peft_model.named_parameters() if any([k in n for k in trainable_params.split(",")])]
pass

In [25]:
peft_model.config.use_cache = False         # required for gradient checkpointing
peft_model.enable_input_require_grads()     # required for gradient checkpointing
peft_model.gradient_checkpointing_enable()  # enable gradient checkpointing

In [26]:
OUTPUT_DIR = "output_dir"

In [27]:
from dataclasses import dataclass, field

@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [46]:
# from accelerate.utils import DistributedType

@dataclass
class TrainingArguments(transformers.TrainingArguments):
    cache_dir: Optional[str] = field(default=None)
    optim: str = field(default="adamw_torch")
    model_max_length: int = field(
        default=8192 * 4,
        metadata={"help": "Maximum sequence length. Sequences will be right padded (and possibly truncated)."},
    )
    use_flash_attn: bool = field(
        default=True,
        metadata={"help": "Whether use flash attention for training."},
    )
    use_full_attn: bool = field(
        default=False,
        metadata={"help": "Whether to use plain, full-attention for training."},
    )
    low_rank_training: bool = field(
        default=True,
        metadata={"help": "Whether use low rank adaptation for training."},
    )
    trainable_params: str = field(
        default="embed,norm",
        metadata={"help": "Additional trainable parameters except LoRA weights, if low rank training."},
    )
    
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    bf16=True,
    use_flash_attn=True,
    low_rank_training=True,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="no",
    # save_strategy="steps",
    # save_steps=1,
    save_total_limit=2,
    learning_rate=2e-5,
    weight_decay=0.0,
    warmup_steps=20,
    lr_scheduler_type="constant_with_warmup",
    logging_steps=1,
    deepspeed="ds_configs/stage2.json",
    tf32=True,
    report_to=['tensorboard'],
)
# training_args.distributed_state.distributed_type = DistributedType.DEEPSPEED
training_args

TrainingArguments(output_dir='output_dir', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=False, evaluation_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=8, eval_accumulation_steps=None, eval_delay=0, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5, max_steps=-1, lr_scheduler_type=<SchedulerType.CONSTANT_WITH_WARMUP: 'constant_with_warmup'>, warmup_ratio=0.0, warmup_steps=20, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='output_dir/runs/May08_19-40-21_g21-52937519-d04a-4a55-bbfa-29fa42618993', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=1, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=5

In [47]:
trainer = Trainer(
    model=peft_model, 
    tokenizer=tokenizer, 
    args=training_args, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset, 
    data_collator=data_collator,
)
trainer.train()
trainer.save_state()
trainer.save_model(output_dir=training_args.output_dir)

/home/jupyter/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(

  0%|          | 0/450 [00:00<?, ?it/s]
                                               
  0%|          | 1/450 [00:03<27:03,  3.62s/it]

{'loss': 1.3015, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}



                                               
  0%|          | 2/450 [00:07<27:28,  3.68s/it]

{'loss': 1.4281, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}



                                               
  1%|          | 3/450 [00:10<27:09,  3.65s/it]

...

{'loss': 1.1439, 'learning_rate': 2e-05, 'epoch': 4.98}



                                               t]
100%|█████████▉| 449/450 [29:59<00:04,  4.13s/it]

{'loss': 1.2452, 'learning_rate': 2e-05, 'epoch': 4.99}



                                               t]
  0%|          | 1/450 [30:54<27:06,  3.62s/it]  
                                               t]
100%|██████████| 450/450 [30:02<00:00,  4.01s/it]
/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: cd0e18bc-21d1-4bc6-b50e-eef032bb5220)') - silently ignoring the lookup for the file config.json in meta-llama/Llama-2-7b-hf.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in meta-llama/Llama-2-7b-hf - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.3703, 'learning_rate': 2e-05, 'epoch': 5.0}
{'train_runtime': 1802.7215, 'train_samples_per_second': 1.997, 'train_steps_per_second': 0.25, 'train_loss': 1.2846886356671652, 'epoch': 5.0}


### Generating answer with tuned model

In [66]:
# 720 train, 720 eval
prefix_len = np.sum(np.array(test_dataset[9]["labels"]) == IGNORE_INDEX)
prefix_tokens = test_dataset[9]["input_ids"][:prefix_len]
peft_model.eval()
generated = peft_model.generate(prefix_tokens.reshape((1, -1)).to(device))
generated_text = tokenizer.decode(generated.to('cpu').flatten())
print(generated_text)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is a diploma text. Your task is to generate abstract of this diploma.

### Input:
Санкт-Петербургский государственный университет


АВРАМЕНКО Полина Андреевна
Выпускная квалификационная работа
Веб-туны как часть южнокорейской культуры в XXI веке (на примере романтических историй) 
Уровень образования: магистратура
Направление 58.04.01 «Востоковедение и африканистика»
Основная образовательная программа BM.5808 «Культура народов Азии и Африки (с изучением языков Азии и Африки)»


Научный руководитель:
доцент, Кафедра корееведения, Санкт-Петербургский государственный университет Гурьева Анастасия Александровна

Рецензент:
приглашенный преподаватель, Кафедра корееведения, Санкт-Петербургская школа социальных наук и востоковедения,
доцент, Санкт-Петербургский филиал федерального государственного автономного 

In [71]:
# 720 train, 720 eval
# inference train sample
peft_model.eval()
prefix_len = np.sum(np.array(train_dataset[9]["labels"]) == IGNORE_INDEX)
prefix_tokens = train_dataset[9]["input_ids"][:prefix_len]
generated = peft_model.generate(prefix_tokens.reshape((1, -1)).to(device))
generated_text = tokenizer.decode(generated.to('cpu').flatten())
print(generated_text)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is a diploma text. Your task is to generate abstract of this diploma.

### Input:
САНКТ-ПЕТЕРБУРГСКИЙ ГОСУДАРСТВЕННЫЙ
УНИВЕРСИТЕТ

Мануэль Антонио Агилар Ривера
Выпускная квалификационная работа
«Multiplication Operators in Cauchy-de Branges Spaces»
Уровень образования: магистратура
Направление 01.04.01 “Математика”
Основная образовательная программа BM.5832.2019
“Современная математика”
Научный руководитель:
Профессор, математико-механический факультет СПбГУ,
доктор физ.-мат. наук, профессор РАН
Баранов Антон Дмитриевич.
Рецензент:
Профессор, Факультет математики,
Автономный университет Мадрида, кандидат физ.-мат. наук
Якубович Дмитрий Владимирович
Санкт-Петербург
2021

Contents
Contents

2

Cauchy–de Branges spaces
3
Cauchy–de Branges spaces as Reproducing Kernel Hilbert spaces . 4
The Division Prope

### View test vs train

In [69]:
# 720 train, 720 eval
# prefix_len = np.sum(np.array(test_dataset[9]["labels"]) == IGNORE_INDEX)
prefix_tokens = test_dataset[9]["input_ids"] #[:prefix_len]
# peft_model.eval()
# generated = peft_model.generate(prefix_tokens.reshape((1, -1)).to(device))
generated_text = tokenizer.decode(prefix_tokens) # generated.to('cpu').flatten())
print(generated_text)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is a diploma text. Your task is to generate abstract of this diploma.

### Input:
Санкт-Петербургский государственный университет


АВРАМЕНКО Полина Андреевна
Выпускная квалификационная работа
Веб-туны как часть южнокорейской культуры в XXI веке (на примере романтических историй) 
Уровень образования: магистратура
Направление 58.04.01 «Востоковедение и африканистика»
Основная образовательная программа BM.5808 «Культура народов Азии и Африки (с изучением языков Азии и Африки)»


Научный руководитель:
доцент, Кафедра корееведения, Санкт-Петербургский государственный университет Гурьева Анастасия Александровна

Рецензент:
приглашенный преподаватель, Кафедра корееведения, Санкт-Петербургская школа социальных наук и востоковедения,
доцент, Санкт-Петербургский филиал федерального государственного автономного 

In [70]:
# 720 train, 720 eval
# prefix_len = np.sum(np.array(test_dataset[9]["labels"]) == IGNORE_INDEX)
prefix_tokens = train_dataset[9]["input_ids"] #[:prefix_len]
# peft_model.eval()
# generated = peft_model.generate(prefix_tokens.reshape((1, -1)).to(device))
generated_text = tokenizer.decode(prefix_tokens) # generated.to('cpu').flatten())
print(generated_text)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Below is a diploma text. Your task is to generate abstract of this diploma.

### Input:
САНКТ-ПЕТЕРБУРГСКИЙ ГОСУДАРСТВЕННЫЙ
УНИВЕРСИТЕТ

Мануэль Антонио Агилар Ривера
Выпускная квалификационная работа
«Multiplication Operators in Cauchy-de Branges Spaces»
Уровень образования: магистратура
Направление 01.04.01 “Математика”
Основная образовательная программа BM.5832.2019
“Современная математика”
Научный руководитель:
Профессор, математико-механический факультет СПбГУ,
доктор физ.-мат. наук, профессор РАН
Баранов Антон Дмитриевич.
Рецензент:
Профессор, Факультет математики,
Автономный университет Мадрида, кандидат физ.-мат. наук
Якубович Дмитрий Владимирович
Санкт-Петербург
2021

Contents
Contents

2

Cauchy–de Branges spaces
3
Cauchy–de Branges spaces as Reproducing Kernel Hilbert spaces . 4
The Division Prope

## Infer learnt model with raw & compare to the target
- raw — model
- learnt — peft_model
- target — abstract

In [79]:
test_df = pd.read_csv("/home/jupyter/mnt/datasets/diplomas/russian_dataset/russian_dataset_test.csv")
test_df.head()

,id,year,diploma,abstract,study_field,degree,original_diploma_extension
0,41453,2023,Санкт-Петербургский государственный университе...,Абдуллаев Ш.У. тема диссертации: «Роль политик...,INTERNATIONAL RELATIONS,MASTER'S STUDIES,.doc
1,43790,2023,Санкт-Петербургский государственный университе...,Выпускная квалификационная работа посвящена из...,POLITICAL SCIENCE,BACHELOR STUDIES,.docx
2,41165,2023,Санкт-Петербургский государственный университе...,В данной работе описывается реализация обобщен...,MATHEMATICS AND MECHANICS,BACHELOR STUDIES,.pdf
3,42349,2023,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \nСАНКТ-ПЕТ...,В выпускной квалификационной работе раскрывает...,SOCIOLOGY,BACHELOR STUDIES,.docx
4,40166,2023,Федеральное государственное бюджетное образова...,На сегодняшний день наблюдается высокий спрос ...,MANAGEMENT,BACHELOR STUDIES,.docx


In [82]:
with open(REPOSITOTY_DIR_PATH.joinpath("src/notebooks/junk/mcs_df_human_filled_processed.json"), "r") as f:
    asessors_questions = json.load(f) 

In [86]:
ids = [int(x['meta']['id']) for x in asessors_questions]

In [91]:
diploma_prefix_len=1000

In [141]:
def get_learnt_model_result(prefix_len, prefix_tokens):
    peft_model.eval()
    generated = peft_model.generate(prefix_tokens.reshape((1, -1)).to(device))
    generated_continue = tokenizer.decode(generated.to('cpu').flatten()[prefix_len:])
    return generated_continue
    # generated_text_all = tokenizer.decode(generated.to('cpu').flatten())
    # return generated_text_all, generated_coninue

In [142]:
def get_prefix_len_and_tokens(row):
    prompt_input_diploma = PROMPT_DICT["prompt_input_diploma_special"]
    source = prompt_input_diploma.format(input=row["diploma"][:diploma_prefix_len])

    target = f"{row['abstract']}{tokenizer.eos_token}"

    data_dict = preprocess([source], [target], tokenizer)
    
    prefix_len = np.sum(np.array(data_dict["labels"][0]) == IGNORE_INDEX)
    # print(prefix_len)
    prefix_tokens = data_dict["input_ids"][0][:prefix_len]
    # print(prefix_tokens)
    return prefix_len, prefix_tokens

In [143]:
def get_raw_model_result(prefix_len, prefix_tokens):
    model.eval()
    generated = model.generate(prefix_tokens.reshape((1, -1)).to(device))
    generated_continue = tokenizer.decode(generated.to('cpu').flatten()[prefix_len:])
    return generated_continue
    # generated_text_all = tokenizer.decode(generated.to('cpu').flatten())
    # return generated_text_all, generated_coninue

In [144]:
new_rows = []
for _, row in tqdm(test_df.iloc[ids].iterrows(), total=len(ids), desc="Rows..."):
    new_row = copy.deepcopy(row)
    prefix_len, prefix_tokens = get_prefix_len_and_tokens(row)
    new_row["raw_model"] = get_raw_model_result(prefix_len, prefix_tokens)
    new_row["learnt"] = get_learnt_model_result(prefix_len, prefix_tokens)
    new_rows.append(new_row)
new_df = pd.DataFrame(new_rows)
new_df.head()

Rows...:   0%|          | 0/70 [00:00<?, ?it/s]

,id,year,diploma,abstract,study_field,degree,original_diploma_extension,raw_model,learnt
12,45042,2023,АЙВАЗЬЯН Аршак Владимирович\nВыпускная квалифи...,В этой работе мы строим правую трансферную мод...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В работе рассматривается модельная структура н...,В данной работе рассматривается модельная стру...
25,45043,2023,Санкт-Петербургский государственный университе...,"Пусть 𝐾 выпуклое тело в ℝ^𝑛. Определим 𝑑𝑛,𝑛−1(...",MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,Плотность решетки трансляций - это минимальная...,В работе рассматриваются плотности решеток тра...
37,45044,2023,Санкт-Петербургский государственный университе...,Работа посвящена повышению производительности ...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В данной работе представлены результаты исслед...,В работе рассматривается задача булевой выполн...
101,45046,2023,Санкт-Петербургский государственный университе...,В работе мы обобщаем результаты об энергии нат...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В работе рассматриваются классы случайных проц...,В данной работе рассматривается энергетически-...
152,45047,2023,Санкт–Петербургский государственный университе...,В рамках данной работы рассматривается подход ...,MATHEMATICS AND COMPUTER SCIENCE,BACHELOR STUDIES,.pdf,В данной работе рассматривается задача настраи...,В данной работе рассматривается задача добавле...


In [145]:
new_df.to_csv(ARTIFACTS_DIR_PATH.joinpath("diplomas_abstracts/mcs_raw_learnt_abstract.csv"))

## Try to save & load learnt model

In [146]:
from huggingface_hub import notebook_login
notebook_login()

In [147]:
model_id = "nvdenisov2002/llama-longLoRA-v1"

In [ ]:
peft_model.push_to_hub(model_id)

In [ ]:
trainer.push_to_hub(model_id)

In [150]:
another_peft_config = PeftConfig.from_pretrained(model_id)
another_peft_model = PeftModel.from_pretrained(model, model_id)

In [151]:
row

id                                                                        45137
year                                                                       2023
diploma                       Санкт–Петербургский государственный университе...
abstract                      В данной работе представлены реализация фронте...
study_field                                    MATHEMATICS AND COMPUTER SCIENCE
degree                                                         BACHELOR STUDIES
original_diploma_extension                                                 .pdf
Name: 1238, dtype: object

In [152]:
prefix_len, prefix_tokens = get_prefix_len_and_tokens(row)
learnt_old = get_learnt_model_result(prefix_len, prefix_tokens)

In [153]:
another_peft_model.eval()
generated = another_peft_model.generate(prefix_tokens.reshape((1, -1)).to(device))
generated_continue = tokenizer.decode(generated.to('cpu').flatten()[prefix_len:])
learnt_new = generated_continue

In [155]:
learnt_old == learnt_new

False

In [156]:
learnt_old

'В работе рассмотрена задача создания персонажа для игры в жанре «космическая охота на разумных существ». Разработана архитектура мобильного приложения для создания персонажа, которая состоит из 3 модулей: Front-end, Back-end и сервис. В Front-end модуле представлены экраны, которые позволяют пользователю создавать персонажа. В Back-end модуле реализован сервис, который позволяет пользователю сохранить созданный персонаж на сервере и использовать его в игре. В Back-end модуле также реализован сервис, который позволяет пользователю изменить свой персонаж, а также сервис, который позволяет пользователю загрузить созданный им персонаж на сервер.</s>'

In [158]:
learnt_old_2 = get_learnt_model_result(prefix_len, prefix_tokens)
learnt_old_2

'В работе рассмотрена задача создания персонажа для онлайн-игры в жанре массовой многопользовательской ролевой игры. Рассматривается архитектура приложения, применяемые технологии, а также реализованные экраны.</s>'

In [157]:
learnt_new

'В работе рассматривается проблема создания персонажа в онлайн-игре. На основе анализа существующих решений и анализа потребностей игроков была разработана архитектура мобильного приложения, реализованного на основе фреймворка Flutter. В результате было создано мобильное приложение для создания персонажа в онлайн-игре.</s>'